# Using the Estimator API 

In [10]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

dataset = pd.read_csv('C:/Users/ASUS/Desktop/500.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

feature_cols = [tf.feature_column.numeric_column("X",shape=[4* 1])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units=[300,100], n_classes=10,
                                     feature_columns=feature_cols)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, num_epochs=40, batch_size=50, shuffle=True)
dnn_clf.train(input_fn=input_fn)

print('eval_results:\n',eval_results)

import gym
import time
env = gym.make('CartPole-v0') # 环境导入
env.reset()
act = 0
total = 0
start = time.time()
for i in range(1000):
    env.render()
    observation, reward, done, info = env.step(act) # 随便动一动~~~
    #print(i,":","action",act,"reward",reward)
    act_pred = []
    act_pred.append(observation)
    
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": np.array(act_pred)}, y=np.array([0]), shuffle=False)#
    
    y_pred_iter = dnn_clf.predict(input_fn=test_input_fn)
    y_pred = list(y_pred_iter)
    act = y_pred[0]['class_ids'][0]
    print("i:",i,"act",act)
    if act>=0.5:
        act = 1
    else:
        act = 0
    total = total + reward
    if done == 1:
        print("stop after",i+1,"steps\n","the total reward is",total)
        break
end = time.time()
print("Execution Time: ", end - start)     
env.close()

#y_pred_iter = dnn_clf.predict(input_fn=test_input_fn)
#y_pred = list(y_pred_iter)
#print('y_pred:\n',y_pred[0]['class_ids'][0])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ASUS\\AppData\\Local\\Temp\\tmpk6gbj8j7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000221D5032F48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Ca

# Using plain TensorFlow

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

n_inputs = 4*1  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

dataset = pd.read_csv('C:/Users/ASUS/Desktop/500.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    print("loss:",loss)
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 40
batch_size = 50

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

import gym
import time
env = gym.make('CartPole-v0') # 环境导入
env.reset()
act = 0
total = 0
start = time.time()
for i in range(1000):
    env.render()
    observation, reward, done, info = env.step(act) # 随便动一动~~~
    #print(i,":","action",act,"reward",reward)
    act_pred = []
    act_pred.append(observation)
    with tf.Session() as sess:
        saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
        X_new_scaled = act_pred
        Z = logits.eval(feed_dict={X: X_new_scaled})
        y_pred = np.argmax(Z, axis=1)
    act = y_pred[0]
    print("i:",i,"act",act)
    if act>=0.5:
        act = 1
    else:
        act = 0
    total = total + reward
    if done == 1:
        print("stop after",i+1,"steps\n","the total reward is",total)
        break
end = time.time()
print("Execution Time: ", end - start)     
env.close()

loss: Tensor("loss_6/loss:0", shape=(), dtype=float32)
0 Batch accuracy: 0.61538464 Val accuracy: 0.61290324
1 Batch accuracy: 0.5769231 Val accuracy: 0.625
2 Batch accuracy: 0.6730769 Val accuracy: 0.66129035
3 Batch accuracy: 0.8076923 Val accuracy: 0.6854839
4 Batch accuracy: 0.6730769 Val accuracy: 0.70564514
5 Batch accuracy: 0.84615386 Val accuracy: 0.73790324
6 Batch accuracy: 0.88461536 Val accuracy: 0.75403225
7 Batch accuracy: 0.84615386 Val accuracy: 0.766129
8 Batch accuracy: 0.78846157 Val accuracy: 0.7701613
9 Batch accuracy: 0.7307692 Val accuracy: 0.8145161
10 Batch accuracy: 0.88461536 Val accuracy: 0.8346774
11 Batch accuracy: 0.8076923 Val accuracy: 0.8346774
12 Batch accuracy: 0.8076923 Val accuracy: 0.858871
13 Batch accuracy: 0.96153843 Val accuracy: 0.8548387
14 Batch accuracy: 0.78846157 Val accuracy: 0.87096775
15 Batch accuracy: 0.90384614 Val accuracy: 0.858871
16 Batch accuracy: 0.88461536 Val accuracy: 0.87096775
17 Batch accuracy: 0.8269231 Val accuracy: 0

# Using dense() instead of neuron_layer()

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

n_inputs = 4*1  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

dataset = pd.read_csv('C:/Users/ASUS/Desktop/500.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu,reuse=True)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu,reuse=True)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs",reuse=True)
    y_proba = tf.nn.softmax(logits)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
n_batches = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")
    
import gym
import time
env = gym.make('CartPole-v0') # 环境导入
env.reset()
act = 0
total = 0
start = time.time()
for i in range(1000):
    env.render()
    observation, reward, done, info = env.step(act) # 随便动一动~~~
    #print(i,":","action",act,"reward",reward)
    act_pred = []
    act_pred.append(observation)
    with tf.Session() as sess:
        saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
        X_new_scaled = act_pred
        Z = logits.eval(feed_dict={X: X_new_scaled})
        y_pred = np.argmax(Z, axis=1)
    act = y_pred[0]
    print("i:",i,"act",act)
    if act>=0.5:
        act = 1
    else:
        act = 0
    total = total + reward
    if done == 1:
        print("stop after",i+1,"steps\n","the total reward is",total)
        break
end = time.time()
print("Execution Time: ", end - start)     
env.close()

0 Batch accuracy: 0.61538464 Validation accuracy: 0.66129035
1 Batch accuracy: 0.75 Validation accuracy: 0.6814516
2 Batch accuracy: 0.63461536 Validation accuracy: 0.6854839
3 Batch accuracy: 0.59615386 Validation accuracy: 0.67741936
4 Batch accuracy: 0.59615386 Validation accuracy: 0.66935486
5 Batch accuracy: 0.6730769 Validation accuracy: 0.66935486
6 Batch accuracy: 0.6923077 Validation accuracy: 0.66935486
7 Batch accuracy: 0.63461536 Validation accuracy: 0.6733871
8 Batch accuracy: 0.78846157 Validation accuracy: 0.66935486
9 Batch accuracy: 0.65384614 Validation accuracy: 0.66935486
10 Batch accuracy: 0.71153843 Validation accuracy: 0.6733871
11 Batch accuracy: 0.7307692 Validation accuracy: 0.67741936
12 Batch accuracy: 0.53846157 Validation accuracy: 0.6854839
13 Batch accuracy: 0.8076923 Validation accuracy: 0.6895161
14 Batch accuracy: 0.71153843 Validation accuracy: 0.67741936
15 Batch accuracy: 0.7307692 Validation accuracy: 0.6733871
16 Batch accuracy: 0.5 Validation ac

KeyboardInterrupt: 

# Batch Normalization

In [1]:
from functools import partial
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

if __name__ == '__main__':
    n_inputs = 4 * 1
    n_hidden1 = 300
    n_hidden2 = 100
    n_outputs = 10
    
    dataset = pd.read_csv('C:/Users/ASUS/Desktop/500.csv')
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

    #mnist = input_data.read_data_sets("/tmp/data/")

    batch_norm_momentum = 0.9
    learning_rate = 0.01

    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = 'X')
    y = tf.placeholder(tf.int64, shape=None, name = 'y')
    training = tf.placeholder_with_default(False, shape=(), name = 'training')#给Batch norm加一个placeholder

    with tf.name_scope("dnn"):
        he_init = tf.contrib.layers.variance_scaling_initializer()
        #对权重的初始化

        my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training = training,
            momentum = batch_norm_momentum
        )

        my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer = he_init
        )

        hidden1 = my_dense_layer(X ,n_hidden1 ,name = 'hidden1',reuse=tf.AUTO_REUSE)
        bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
        hidden2 = my_dense_layer(bn1, n_hidden2, name = 'hidden2',reuse=tf.AUTO_REUSE)
        bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
        logists_before_bn = my_dense_layer(bn2, n_outputs, name = 'outputs',reuse=tf.AUTO_REUSE)
        logists = my_batch_norm_layer(logists_before_bn)

    with tf.name_scope('loss'):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits= logists)
        loss = tf.reduce_mean(xentropy, name = 'loss')

    with tf.name_scope('train'):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)

    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logists, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    n_epoches = 50
    batch_size = 200
# 注意：由于我们使用的是 tf.layers.batch_normalization() 而不是 tf.contrib.layers.batch_norm()（如本书所述），
# 所以我们需要明确运行批量规范化所需的额外更新操作（sess.run([ training_op，extra_update_ops], ...)。
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epoches):
            for iteraton in range(batch_size):
                X_batch = X_train
                y_batch = y_train
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
        save_path = saver.save(sess, "./my_model_final.ckpt")
    
    import gym
    import time
    env = gym.make('CartPole-v0') # 环境导入
    env.reset()
    act = 0
    total = 0
    start = time.time()
    for i in range(1000):
        env.render()
        observation, reward, done, info = env.step(act) # 随便动一动~~~
        #print(i,":","action",act,"reward",reward)
        act_pred = []
        act_pred.append(observation)
        with tf.Session() as sess:
            saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
            X_new_scaled = np.array(act_pred)
            Z = accuracy.eval(feed_dict={X: X_new_scaled,y:np.array([0])})
            #y_pred = np.argmax(Z, axis=1)
        act = Z
        print("i:",i,"act",act)
        if act>=0.5:
            act = 1
        else:
            act = 0
        total = total + reward
        if done == 1:
            print("stop after",i+1,"steps\n","the total reward is",total)
            break
    end = time.time()
    print("Execution Time: ", end - start)     
    env.close()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
0 Batch accuracy: 0.92439514 Validation accuracy: 0.9233871
1 Batch accuracy: 0.9405242 Validation accuracy: 0.9395161
2 Batch accuracy: 0.9445565 Validation accuracy: 0.9314516
3 Batch accuracy: 0.9465726 Validation accuracy: 0.92741936
4 Batch accuracy: 0.9465726 Validation accu

# Faster Optimizers：Adam

In [1]:
from functools import partial
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

if __name__ == '__main__':
    n_inputs = 4 * 1
    n_hidden1 = 300
    n_hidden2 = 100
    n_outputs = 10
    
    dataset = pd.read_csv('C:/Users/ASUS/Desktop/500.csv')
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

    #mnist = input_data.read_data_sets("/tmp/data/")

    batch_norm_momentum = 0.9
    learning_rate = 0.01

    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = 'X')
    y = tf.placeholder(tf.int64, shape=None, name = 'y')
    training = tf.placeholder_with_default(False, shape=(), name = 'training')#给Batch norm加一个placeholder

    with tf.name_scope("dnn"):
        he_init = tf.contrib.layers.variance_scaling_initializer()
        #对权重的初始化

        my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training = training,
            momentum = batch_norm_momentum
        )

        my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer = he_init
        )

        hidden1 = my_dense_layer(X ,n_hidden1 ,name = 'hidden1',reuse=tf.AUTO_REUSE)
        bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
        hidden2 = my_dense_layer(bn1, n_hidden2, name = 'hidden2',reuse=tf.AUTO_REUSE)
        bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
        logists_before_bn = my_dense_layer(bn2, n_outputs, name = 'outputs',reuse=tf.AUTO_REUSE)
        logists = my_batch_norm_layer(logists_before_bn)

    with tf.name_scope('loss'):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits= logists)
        loss = tf.reduce_mean(xentropy, name = 'loss')

    with tf.name_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss)

    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logists, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    n_epoches = 20
    batch_size = 200
# 注意：由于我们使用的是 tf.layers.batch_normalization() 而不是 tf.contrib.layers.batch_norm()（如本书所述），
# 所以我们需要明确运行批量规范化所需的额外更新操作（sess.run([ training_op，extra_update_ops], ...)。
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epoches):
            for iteraton in range(batch_size):
                X_batch = X_train
                y_batch = y_train
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
        save_path = saver.save(sess, "./my_model_final.ckpt")
    
    import gym
    import time
    env = gym.make('CartPole-v0') # 环境导入
    env.reset()
    act = 0
    total = 0
    start = time.time()
    for i in range(1000):
        env.render()
        observation, reward, done, info = env.step(act) # 随便动一动~~~
        #print(i,":","action",act,"reward",reward)
        act_pred = []
        act_pred.append(observation)
        with tf.Session() as sess:
            saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
            X_new_scaled = np.array(act_pred)
            Z = accuracy.eval(feed_dict={X: X_new_scaled,y:np.array([0])})
            #y_pred = np.argmax(Z, axis=1)
        act = Z
        print("i:",i,"act",act)
        if act>=0.5:
            act = 1
        else:
            act = 0
        total = total + reward
        if done == 1:
            print("stop after",i+1,"steps\n","the total reward is",total)
            break
    end = time.time()
    print("Execution Time: ", end - start)     
    env.close()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
0 Batch accuracy: 0.96370965 Validation accuracy: 0.96370965
1 Batch accuracy: 0.9455645 Validation accuracy: 0.9395161
2 Batch accuracy: 0.9606855 Validation accuracy: 0.95564514
3 Batch accuracy: 0.98084676 Validation accuracy: 0.96370965
4 Batch accuracy: 0.98689514 Validation 

# Avoiding Overfitting Through Regularization

In [1]:
from functools import partial
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

if __name__ == '__main__':
    n_inputs = 4 * 1
    n_hidden1 = 300
    n_hidden2 = 100
    n_outputs = 10
    
    dataset = pd.read_csv('C:/Users/ASUS/Desktop/500.csv')
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

    #mnist = input_data.read_data_sets("/tmp/data/")

    batch_norm_momentum = 0.9
    learning_rate = 0.01
    scale = 0.001

    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = 'X')
    y = tf.placeholder(tf.int64, shape=None, name = 'y')
    training = tf.placeholder_with_default(False, shape=(), name = 'training')#给Batch norm加一个placeholder
    
    my_dense_layer = partial(
        tf.layers.dense, activation=tf.nn.relu,
        kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))

    with tf.name_scope("dnn"):
        hidden1 = my_dense_layer(X, n_hidden1, name="hidden1",reuse=tf.AUTO_REUSE)
        hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2",reuse=tf.AUTO_REUSE)
        logits = my_dense_layer(hidden2, n_outputs, activation=None,
                                name="outputs",reuse=tf.AUTO_REUSE)

    with tf.name_scope("loss"):                                     # not shown in the book
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(  # not shown
            labels=y, logits=logits)                                # not shown
        base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")   # not shown
        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        loss = tf.add_n([base_loss] + reg_losses, name="loss")

    with tf.name_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss)

    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    n_epoches = 20
    batch_size = 200
# 注意：由于我们使用的是 tf.layers.batch_normalization() 而不是 tf.contrib.layers.batch_norm()（如本书所述），
# 所以我们需要明确运行批量规范化所需的额外更新操作（sess.run([ training_op，extra_update_ops], ...)。
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epoches):
            for iteraton in range(batch_size):
                X_batch = X_train
                y_batch = y_train
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_valid = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
        save_path = saver.save(sess, "./my_model_final.ckpt")
    
    import gym
    import time
    env = gym.make('CartPole-v0') # 环境导入
    env.reset()
    act = 0
    total = 0
    start = time.time()
    for i in range(1000):
        env.render()
        observation, reward, done, info = env.step(act) # 随便动一动~~~
        #print(i,":","action",act,"reward",reward)
        act_pred = []
        act_pred.append(observation)
        with tf.Session() as sess:
            saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
            X_new_scaled = np.array(act_pred)
            Z = accuracy.eval(feed_dict={X: X_new_scaled,y:np.array([0])})
            #y_pred = np.argmax(Z, axis=1)
        act = Z
        print("i:",i,"act",act)
        if act>=0.5:
            act = 1
        else:
            act = 0
        total = total + reward
        if done == 1:
            print("stop after",i+1,"steps\n","the total reward is",total)
            break
    end = time.time()
    print("Execution Time: ", end - start)     
    env.close()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
0 Batch accuracy: 0.9435484 Validation accuracy: 0.9354839
1 Batch accuracy: 0.95060486 Validation accuracy: 0.9395161
2 Batch accuracy: 0.9546371 Validation accuracy: 0.9596774
3 Batch accuracy: 0.95665324 Validation accuracy: 0.96370965
4 Batch accuracy: 0.9546371 Validation accuracy: 0.96370965
5 Batch accuracy: 0.9546371 Validation accuracy: 0.96370965
6 Batch accuracy: 0.9546371 Validation accuracy: 0.9596774
7 Batch accuracy: 0.95665324 Validation accuracy: 0.96370965
8 Batch accuracy: 0.9556451

In [6]:
import tensorflow as tf 
from tensorflow.contrib.layers import fully_connected
# 1. 声明神经网络结构 
n_inputs = 4  # == env.observation_space.shape[0] 
n_hidden = 4  # 这只是个简单的测试，不需要过多的隐藏层
n_outputs = 1 # 只输出向左加速的概率
initializer = tf.contrib.layers.variance_scaling_initializer()
# 2. 建立神经网络 
X = tf.placeholder(tf.float32, shape=[None, n_inputs]) 
hidden = fully_connected(X, n_hidden, activation_fn=tf.nn.elu,weights_initializer=initializer) # 隐层激活函数使用指数线性函数                
logits = fully_connected(hidden, n_outputs, activation_fn=None,weights_initializer=initializer)                   
outputs = tf.nn.sigmoid(logits)
# 3. 在概率基础上随机选择动作
p_left_and_right = tf.concat(axis=1, values=[outputs, 1 - outputs]) 
action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)
init = tf.global_variables_initializer() 

Instructions for updating:
Use `tf.random.categorical` instead.


# PG算法

In [ ]:
"""
Policy Gradient, Reinforcement Learning.
The cart pole example
View more on my tutorial page: https://morvanzhou.github.io/tutorials/
Using:
Tensorflow: 1.0
gym: 0.8.0
"""
import numpy as np
import tensorflow as tf

# reproducible
np.random.seed(1)
tf.set_random_seed(1)


class PolicyGradient:
    # 初始化
    def __init__(self,n_actions,n_features,learning_rate=0.01,reward_decay=0.95,output_graph=False,):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay

        self.ep_obs, self.ep_as, self.ep_rs = [], [], []

        self._build_net()   # 建立神经网络

        self.sess = tf.Session()

        if output_graph:    # 是否输出 tensorboard 文件
            # $ tensorboard --logdir=logs
            # http://0.0.0.0:6006/
            # tf.train.SummaryWriter soon be deprecated, use following
            tf.summary.FileWriter("logs/", self.sess.graph)

        self.sess.run(tf.global_variables_initializer())

    # 建立 policy gradient 神经网络
    def _build_net(self):
        with tf.name_scope('inputs'):
            self.tf_obs = tf.placeholder(tf.float32, [None, self.n_features], name="observations")
            self.tf_acts = tf.placeholder(tf.int32, [None, ], name="actions_num")
            self.tf_vt = tf.placeholder(tf.float32, [None, ], name="actions_value")
        # fc1
        layer = tf.layers.dense(
            inputs=self.tf_obs,
            units=10,
            activation=tf.nn.tanh,  # tanh activation
            kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.3),
            bias_initializer=tf.constant_initializer(0.1),
            name='fc1'
        )
        # fc2
        all_act = tf.layers.dense(
            inputs=layer,
            units=self.n_actions,
            activation=None,
            kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.3),
            bias_initializer=tf.constant_initializer(0.1),
            name='fc2'
        )

        self.all_act_prob = tf.nn.softmax(all_act, name='act_prob')  # use softmax to convert to probability

        with tf.name_scope('loss'):
            # to maximize total reward (log_p * R) is to minimize -(log_p * R), and the tf only have minimize(loss)
            neg_log_prob = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=all_act, labels=self.tf_acts)   # this is negative log of chosen action
            # or in this way:
            # neg_log_prob = tf.reduce_sum(-tf.log(self.all_act_prob)*tf.one_hot(self.tf_acts, self.n_actions), axis=1)
            loss = tf.reduce_mean(neg_log_prob * self.tf_vt)  # reward guided loss

        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(self.lr).minimize(loss)

    # 选行为
    def choose_action(self, observation):
        # 算出所有行为的概率
        prob_weights = self.sess.run(self.all_act_prob, feed_dict={self.tf_obs: observation[np.newaxis, :]})
        # 根据概率选行为
        action = np.random.choice(range(prob_weights.shape[1]), p=prob_weights.ravel())
        return action

    # 存储回合 transition
    def store_transition(self, s, a, r):
        self.ep_obs.append(s)
        self.ep_as.append(a)
        self.ep_rs.append(r)

    # 学习更新参数
    def learn(self):
        # 衰减, 并标准化这回合的 reward
        discounted_ep_rs_norm = self._discount_and_norm_rewards()

        # train on episode
        self.sess.run(self.train_op, feed_dict={
             self.tf_obs: np.vstack(self.ep_obs),  # shape=[None, n_obs]
             self.tf_acts: np.array(self.ep_as),  # shape=[None, ]
             self.tf_vt: discounted_ep_rs_norm,  # shape=[None, ]
        })

        self.ep_obs, self.ep_as, self.ep_rs = [], [], []     # 清空回合 data
        return discounted_ep_rs_norm    # 返回这一回合的 state-action value

    # 衰减回合的 reward
    def _discount_and_norm_rewards(self):
        # 统计一个回合的奖励
        discounted_ep_rs = np.zeros_like(self.ep_rs)
        running_add = 0
        for t in reversed(range(0, len(self.ep_rs))):
            running_add = running_add * self.gamma + self.ep_rs[t]
            discounted_ep_rs[t] = running_add

        # 标准化回合奖励值
        discounted_ep_rs -= np.mean(discounted_ep_rs)
        discounted_ep_rs /= np.std(discounted_ep_rs)
        return discounted_ep_rs
    
import gym
import matplotlib.pyplot as plt
import time

DISPLAY_REWARD_THRESHOLD = 400  # 当回合总reward大于400的时候显示模拟窗口
RENDER = False  # 边训练边显示会拖慢训练速度，我们等程序先学习一段时间

env = gym.make('CartPole-v0')   # 创建 CardPole这个模拟
env.seed(1)     # 创建随机种子
env = env.unwrapped # 取消限制

print(env.action_space) #输出可用的动作
print(env.observation_space)    # 显示可用 state 的 observation
print(env.observation_space.high)  # 显示 observation 最高值
print(env.observation_space.low)   # 显示 observation 最低值

# 定义使用 Policy_gradient 的算法
RL = PolicyGradient(
    n_actions=env.action_space.n,
    n_features=env.observation_space.shape[0],
    learning_rate=0.02,
    reward_decay=0.99,
    # output_graph=True,
)

for i_episode in range(300):

    observation = env.reset()   # 获取回合 i_episode 第一个 observation
    start = time.time()
    sumreward=0
    while True:
        if RENDER: env.render() # 刷新环境

        action = RL.choose_action(observation)  # 选行为

        observation_, reward, done, info = env.step(action) # 获取下一个state

        RL.store_transition(observation, action, reward)    # 存储这一回合的transition
        sumreward=sumreward+reward
        if done:    # 一个回合结束，开始更新参数
            ep_rs_sum = sum(RL.ep_rs)   # 统计每回合的reward

            if 'running_reward' not in globals():
                running_reward = ep_rs_sum
            else:
                running_reward = running_reward * 0.99 + ep_rs_sum * 0.01
            if running_reward > DISPLAY_REWARD_THRESHOLD: RENDER = True     # 判断是否开始模拟
            print("episode:", i_episode, "  reward:", sumreward)
            end = time.time()
            print("Execution Time: ", end - start)
            vt = RL.learn() # 学习参数，输出vt

            if i_episode == 0:  # 画图
                plt.plot(vt)
                plt.xlabel('episode steps')
                plt.ylabel('normalized state-action value')
                plt.show()
            break

        observation = observation_

Discrete(2)
Box(4,)
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
episode: 0   reward: 13.0
Execution Time:  0.019984960556030273


<Figure size 640x480 with 1 Axes>

episode: 1   reward: 75.0
Execution Time:  0.050284624099731445
episode: 2   reward: 35.0
Execution Time:  0.022143125534057617
episode: 3   reward: 24.0
Execution Time:  0.029982328414916992
episode: 4   reward: 22.0
Execution Time:  0.019998550415039062
episode: 5   reward: 53.0
Execution Time:  0.038210391998291016
episode: 6   reward: 25.0
Execution Time:  0.015979528427124023
episode: 7   reward: 64.0
Execution Time:  0.028276920318603516
episode: 8   reward: 58.0
Execution Time:  0.0499727725982666
episode: 9   reward: 13.0
Execution Time:  0.010013580322265625
episode: 10   reward: 68.0
Execution Time:  0.05533480644226074
episode: 11   reward: 28.0
Execution Time:  0.01997208595275879
episode: 12   reward: 101.0
Execution Time:  0.06391525268554688
episode: 13   reward: 26.0
Execution Time:  0.0159757137298584
episode: 14   reward: 60.0
Execution Time:  0.03595709800720215
episode: 15   reward: 25.0
Execution Time:  0.015976905822753906
episode: 16   reward: 24.0
Execution Time

# DQN

In [1]:
import gym
import tensorflow as tf
import numpy as np
import random
from collections import deque
import time
# Hyper Parameters for DQN
GAMMA = 0.9 # discount factor for target Q
INITIAL_EPSILON = 0.5 # starting value of epsilon
FINAL_EPSILON = 0.01 # final value of epsilon
REPLAY_SIZE = 10000 # experience replay buffer size
BATCH_SIZE = 32 # size of minibatch
 
class DQN():
  # DQN Agent
    def __init__(self, env):
        # init experience replay
        self.replay_buffer = deque()
        # init some parameters
        self.time_step = 0
        self.epsilon = INITIAL_EPSILON
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
 
        self.create_Q_network()
        self.create_training_method()
 
        # Init session
        self.session = tf.InteractiveSession()
        self.session.run(tf.initialize_all_variables())
 
    def create_Q_network(self):
        # network weights
        W1 = self.weight_variable([self.state_dim,20])
        b1 = self.bias_variable([20])
        W2 = self.weight_variable([20,self.action_dim])
        b2 = self.bias_variable([self.action_dim])
        # input layer
        self.state_input = tf.placeholder("float",[None,self.state_dim])
        # hidden layers
        h_layer = tf.nn.relu(tf.matmul(self.state_input,W1) + b1)
        # Q Value layer
        self.Q_value = tf.matmul(h_layer,W2) + b2
 
    def create_training_method(self):
        self.action_input = tf.placeholder("float",[None,self.action_dim]) # one hot presentation
        self.y_input = tf.placeholder("float",[None])
        Q_action = tf.reduce_sum(tf.multiply (self.Q_value,self.action_input),reduction_indices = 1)
        self.cost = tf.reduce_mean(tf.square(self.y_input - Q_action))
        self.optimizer = tf.train.AdamOptimizer(0.0001).minimize(self.cost)
 
    def perceive(self,state,action,reward,next_state,done):
        one_hot_action = np.zeros(self.action_dim)
        one_hot_action[action] = 1
        self.replay_buffer.append((state,one_hot_action,reward,next_state,done))
        if len(self.replay_buffer) > REPLAY_SIZE:
            self.replay_buffer.popleft()
 
        if len(self.replay_buffer) > BATCH_SIZE:
            self.train_Q_network()
 
    def train_Q_network(self):
        self.time_step += 1
        # Step 1: obtain random minibatch from replay memory
        minibatch = random.sample(self.replay_buffer,BATCH_SIZE)
        state_batch = [data[0] for data in minibatch]
        action_batch = [data[1] for data in minibatch]
        reward_batch = [data[2] for data in minibatch]
        next_state_batch = [data[3] for data in minibatch]
 
        # Step 2: calculate y
        y_batch = []
        Q_value_batch = self.Q_value.eval(feed_dict={self.state_input:next_state_batch})
        for i in range(0,BATCH_SIZE):
            done = minibatch[i][4]
            if done:
                y_batch.append(reward_batch[i])
            else :
                y_batch.append(reward_batch[i] + GAMMA * np.max(Q_value_batch[i]))
        
        #print(y_batch==Q_value_batch)
        #print(Q_value_batch)
        
        self.optimizer.run(feed_dict={
          self.y_input:y_batch,
          self.action_input:action_batch,
          self.state_input:state_batch
          })
 
    def egreedy_action(self,state):
        Q_value = self.Q_value.eval(feed_dict = {
          self.state_input:[state]
          })[0]
        if random.random() <= self.epsilon:
            return random.randint(0,self.action_dim - 1)
        else:
            return np.argmax(Q_value)
 
        self.epsilon -= (INITIAL_EPSILON - FINAL_EPSILON)/10000
 
    def action(self,state):
        return np.argmax(self.Q_value.eval(feed_dict = {
          self.state_input:[state]
          })[0])
 
    def weight_variable(self,shape):
        initial = tf.truncated_normal(shape)
        return tf.Variable(initial)
 
    def bias_variable(self,shape):
        initial = tf.constant(0.01, shape = shape)
        return tf.Variable(initial)
# ---------------------------------------------------------
# Hyper Parameters
ENV_NAME = 'CartPole-v0'
EPISODE = 1 # Episode limitation
STEP = 250 # Step limitation in an episode
TEST = 10 # The number of experiment test every 100 episode
 
def main():
  # initialize OpenAI Gym env and dqn agent
    env = gym.make(ENV_NAME)
    agent = DQN(env)
 
    for episode in range(EPISODE):
        # initialize task
        state = env.reset()
        # Train
        sumreward=0
        for step in range(STEP):
            action = agent.egreedy_action(state) # e-greedy action for train
            next_state,reward,done,_ = env.step(action)
            # Define reward for agent
            reward_agent = -1 if done else 0.1
            agent.perceive(state,action,reward,next_state,done)
            state = next_state
            sumreward=sumreward+reward
            if done:
                print(sumreward)
                break
        # Test every 100 episodes
        if episode % 100 == 0:
            total_reward = 0
            start = time.time()
            for i in range(TEST):
                state = env.reset()
                for j in range(STEP):
                    env.render()
                    action = agent.action(state) # direct action for test
                    state,reward,done,_ = env.step(action)
                    total_reward += reward
                    if done:
                        end = time.time()
                        print("score:",total_reward)
                        print("Execution Time: ", end - start)
                        break
            ave_reward = total_reward/TEST
            print('episode: ',episode,'Evaluation Average Reward:',ave_reward)
            if ave_reward >= 200:
                break 

if __name__ == '__main__':
    main()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
69.0
score: 200.0
Execution Time:  3.5551085472106934
score: 230.0
Execution Time:  4.055949687957764
score: 430.0
Execution Time:  7.3900134563446045
score: 630.0
Execution Time:  10.72300100326538
score: 830.0
Execution Time:  14.053975582122803
score: 1030.0
Execution Time:  17.39365816116333
score: 1230.0
Execution Time:  20.9050350189209
score: 1316.0
Execution Time:  22.444984674453735
score: 1380.0
Execution Time:  23.50459361076355
score: 1419.0
Execution Time:  24.152978897094727
episode:  0 Evaluation Average Reward: 141.9
